In [1]:
!pip install google.cloud.translate

DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621
  Using cached google_cloud_core-2.4.1-py2.py3-none-any.whl.metadata (2.7 kB)
  Using cached rsa-4.9-py3-none-any.whl.metadata (4.2 kB)
Using cached google_cloud_core-2.4.1-py2.py3-none-any.whl (29 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.7/10.7 MB 6.4 MB/s eta 0:00:00a 0:00:01
Using cached rsa-4.9-py3-none-any.whl (34 kB)
  DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621
  DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are

In [1]:
import os, csv
# public
import nltk
from nltk.translate.bleu_score import sentence_bleu, corpus_bleu
import pandas as pd
from tqdm import tqdm
from google.cloud import translate_v2 as translate

In [2]:
# a method that calls the cloud translator
def translate_text(source: str, target: str, text: str) -> dict:
    """
    Translates text into the target language.
    Target must be an ISO 639-1 language code.
    See https://g.co/cloud/translate/v2/translate-reference#supported_languages
    """

    translate_client = translate.Client()

    if isinstance(text, bytes):
        text = text.decode("utf-8")

    # Text can also be a sequence of strings, in which case this method
    # will return a sequence of results for each text.
    result = translate_client.translate(
        text
        , source_language=source
        , target_language=target
    )
    return result

def process_nan(s):
  if type(s)!=type(''):
    return ''
  return s

In [3]:
src_lan = 'en'
# target language
tgt_lan = 'es'
language = "Spanish"


RESOURCE = ''
DATA =  RESOURCE+'/data/'+language
GOLD_FILE = os.path.join(DATA, 'gold-sentences-'+language+'.tsv')
TRANS_FILE = os.path.join(DATA, 'trans-sentences-'+language+'.tsv')

# DATA = os.path.join(RESOURCE, 'data')
DATA =  RESOURCE+'data'

KEY = os.path.join(RESOURCE, 'key')
# RAW_SENSE_TSV = os.path.join(DATA, 'gold-tokens-English-wSenses.tsv')
# RAW_TOKENS_TSV =  os.path.join(DATA, 'gold-tokens-English-wSenses.tsv')
RAW_SENTENCE_TSV = os.path.join(DATA, 'SemEval2025_EAMT_Samples.tsv')
JSON_GOOGLE_APPLICATION_CREDENTIALS = os.path.join(KEY, 'leafy-winter-436121-s1-c359b02fd0ef.json')
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = JSON_GOOGLE_APPLICATION_CREDENTIALS

# DATA

In [4]:
sentence_df = pd.read_csv(RAW_SENTENCE_TSV, delimiter='\t')
sentence_df.head()
sents = sentence_df["source"].tolist()
print(sentence_df["target"].tolist())
targets = sentence_df["target_locale"].tolist()
sources = sentence_df["source_locale"].tolist()
langs = [ [sources[x],targets[x]] for x in range(len(sources))]
print(langs)
# sents

["['Görünmez Adam hangi türe ait?']", "['Mulheres Apaixonadas dizisi ilk ne zaman yayınlandı?']", '["Sefirin Kızı\'nın kaç sezonu var", "Sefirin Kızı\'nın kaç sezonu var?", \'Sefirin Kızının kaç sezonu var?\']', "['Kedilere Bulaşmayın: İnternette Katil Avı dizisinin ana odak noktası nedir?']", '[\'9. Senfonide kaç hareket var?\', "9. Senfoni\'de kaç bölüm var?", "9. Senfoni\'de kaç hareket vardır?"]', '["Descartes\'ın Yöntem Üzerine Konuşma\'sı hakkında kısa bir açıklama yapabilir misiniz?", "Descartes\'ın Yöntem Üzerine Konuşma\'nın kısa bir tanımını yapabilir misiniz?"]', '["Üç Silahşörler\'in türü nedir?"]', "['Aziz Vasil Katedrali ne zaman inşa edildi?']", '["Maymunlar Gezegeni\'nde olaylar hangi ülkede geçmiştir?", "Maymunlar Gezegeni\'ndeki olaylar hangi ülkede geçmiştir?", \'Maymunlar Gezegenindeki olaylar hangi ülkede geçiyor?\']', '["Türk\'ün satranç oynama yeteneğiyle ilgili herhangi bir söylenti veya tartışma oldu mu?", "Türk\'ün satranç oynama yeteneğiyle ilgili herhangi bi

In [5]:
sids = sentence_df["id"].tolist()
# sids = [i.replace('d', 'i') for i in sids]
sids[:4]

['Q1539509_0', 'Q2475229_0', 'Q79089380_0', 'Q80546046_0']

In [6]:
# len(sids), len(sents)

# TRANSLATE

In [7]:
# get languages
client = translate.Client()
languages = client.get_languages()
languages

[{'language': 'ab', 'name': 'Abkhaz'},
 {'language': 'ace', 'name': 'Acehnese'},
 {'language': 'ach', 'name': 'Acholi'},
 {'language': 'aa', 'name': 'Afar'},
 {'language': 'af', 'name': 'Afrikaans'},
 {'language': 'sq', 'name': 'Albanian'},
 {'language': 'alz', 'name': 'Alur'},
 {'language': 'am', 'name': 'Amharic'},
 {'language': 'ar', 'name': 'Arabic'},
 {'language': 'hy', 'name': 'Armenian'},
 {'language': 'as', 'name': 'Assamese'},
 {'language': 'av', 'name': 'Avar'},
 {'language': 'awa', 'name': 'Awadhi'},
 {'language': 'ay', 'name': 'Aymara'},
 {'language': 'az', 'name': 'Azerbaijani'},
 {'language': 'ban', 'name': 'Balinese'},
 {'language': 'bal', 'name': 'Baluchi'},
 {'language': 'bm', 'name': 'Bambara'},
 {'language': 'bci', 'name': 'Baoulé'},
 {'language': 'ba', 'name': 'Bashkir'},
 {'language': 'eu', 'name': 'Basque'},
 {'language': 'btx', 'name': 'Batak Karo'},
 {'language': 'bts', 'name': 'Batak Simalungun'},
 {'language': 'bbc', 'name': 'Batak Toba'},
 {'language': 'be', 

In [8]:
src_lan, tgt_lan

('en', 'es')

In [19]:
# sample test
file = "/Users/jairiley/Desktop/Research/SemEval2025/data/SemEval2025_EAMT_Samples.tsv"
df = pd.read_csv(file,delimiter="\t")
sentences = []
for x in tqdm(range(len(df['source']))):
    if str(df.iloc[x]["BN Translated Entity"]).split(",")[0] != "nan" and str(df.iloc[x]["BN Translated Entity"]).split(",")[0] != str(df.iloc[x]["GPT4 Entities"]).split(",")[0]:
        
        ent = df.iloc[x]["GPT4 Entities"].split(",")[0]
        sent = df.iloc[x]['source'].replace(ent, "<<>>")
        test = translate_text(langs[x][0], langs[x][1], sent)
    
        ent = df.iloc[x]["BN Translated Entity"].split(",")[0]
        test["translatedText"] = test["translatedText"].replace("&lt;&lt;&gt;&gt;",ent).replace("&#39;","'")
        sentences.append(test)
    else:
        # sent = df.iloc[x]['source'].replace(ent, "<<>>")
        test = translate_text(langs[x][0], langs[x][1], df.iloc[x]['source'])
        sentences.append(test)
        
    # print(sent)
sentences

100%|██████████| 200/200 [01:21<00:00,  2.45it/s]


[{'translatedText': 'Görünmez Adam hangi türe aittir?',
  'input': 'What genre does <<>> belong to?'},
 {'translatedText': 'Mulheres Apaixonadas dizisi ilk ne zaman yayınlandı?',
  'input': 'When was the TV series Mulheres Apaixonadas first aired?'},
 {'translatedText': 'Sefirin Kızı dizisinin kaç sezonu var?',
  'input': 'How many seasons of <<>> are there?'},
 {'translatedText': 'Kedilere Bulaşmayın: İnternette Katil Avı dizisinin ana odağı nedir?',
  'input': 'What is the main focus of the TV series <<>>?'},
 {'translatedText': "9. Senfoni'de kaç hareket var?",
  'input': 'How many movements are there in <<>>?'},
 {'translatedText': "René Descartes' Yöntem Üzerine Söylem'i kısaca açıklayabilir misiniz?",
  'input': "Can you give a brief description of <<>>' Discourse on the Method?"},
 {'translatedText': "Üç Silahşörler'in türü nedir?",
  'input': 'What is the genre of <<>>?'},
 {'translatedText': 'Aziz Vasil Katedrali ne zaman inşa edildi?',
  'input': 'When was <<>> built?'},
 {'t

In [21]:
# translating all the sentences from English to target language
tgt_sents = []
for x in sentences:
    tgt_sents.append(x["translatedText"])
print(tgt_sents)

['Görünmez Adam hangi türe aittir?', 'Mulheres Apaixonadas dizisi ilk ne zaman yayınlandı?', 'Sefirin Kızı dizisinin kaç sezonu var?', 'Kedilere Bulaşmayın: İnternette Katil Avı dizisinin ana odağı nedir?', "9. Senfoni'de kaç hareket var?", "René Descartes' Yöntem Üzerine Söylem'i kısaca açıklayabilir misiniz?", "Üç Silahşörler'in türü nedir?", 'Aziz Vasil Katedrali ne zaman inşa edildi?', 'Maymunlar Cehennemi olayları hangi ülkede gerçekleşti?', "Türk'in satranç oynama yeteneği konusunda herhangi bir söylenti veya tartışma var mıydı?", "Berliner Dom'i nasıl tanımlarsınız?", 'Rusya Devlet Başkanı nasıl seçilir?', 'Çernobil Nükleer Santrali&#39;nde başmühendis yardımcısı olarak görev yapan Anatoli Dyatlov&#39;un görevleri nelerdi?', 'Hangi dini metinlerde Atababalar Mağarası ifadesi geçmektedir?', 'Star Wars evreninde Legend of the Galactic Heroes simgesinin anlamı nedir?', 'Mescid-i Nebevî kaç kişiyi ağırlayabilir?', "Hüzün Kenti'in türü nedir?", "Aleksey Nikolayeviç Romanov Rusya Çare

In [22]:

df["BN Replacement Translation"] = tgt_sents
df.head()

,Label,id,source_locale,target_locale,source,target,metadata,Cloud Translation,GPT4 Translations,Cloud BLEU,GPT4 BLEU,GPT4 Entities,BN Translated Entity,BN Replacement Translation
0,0,Q1539509_0,en,tr,What genre does The Invisible Man belong to?,['Görünmez Adam hangi türe ait?'],{'wikidata_id': 'Q1539509'},Görünmez Adam hangi türe aittir?,Görünmez Adam hangi türe aittir?,0.67,0.67,The Invisible Man,Görünmez Adam,Görünmez Adam hangi türe aittir?
1,1,Q2475229_0,en,tr,When was the TV series Mulheres Apaixonadas fi...,['Mulheres Apaixonadas dizisi ilk ne zaman yay...,{'wikidata_id': 'Q2475229'},Mulheres Apaixonadas dizisi ilk ne zaman yayın...,TV dizisi Mulheres Apaixonadas ilk olarak ne z...,1.00,0.00,Mulheres Apaixonadas,Mulheres Apaixonadas,Mulheres Apaixonadas dizisi ilk ne zaman yayın...
2,2,Q79089380_0,en,tr,How many seasons of The Ambassador's Daughter ...,"[""Sefirin Kızı'nın kaç sezonu var"", ""Sefirin K...",{'wikidata_id': 'Q79089380'},The Ambassador's Daughter dizisinin kaç sezonu...,Elçin'in Kızı dizisinin kaç sezonu var?,0.00,0.00,The Ambassador's Daughter,Sefirin Kızı,Sefirin Kızı dizisinin kaç sezonu var?
3,3,Q80546046_0,en,tr,What is the main focus of the TV series Don't ...,['Kedilere Bulaşmayın: İnternette Katil Avı di...,{'wikidata_id': 'Q80546046'},Kedilerle Uğraşmayın: İnternet Katilini Avlama...,Dont F**k with Cats: Hunting an Internet Kille...,0.00,0.26,Don't F**k with Cats: Hunting an Internet Killer,Kedilere Bulaşmayın: İnternette Katil Avı,Kedilere Bulaşmayın: İnternette Katil Avı dizi...
4,4,Q11989_0,en,tr,How many movements are there in Symphony No. 9?,"['9. Senfonide kaç hareket var?', ""9. Senfoni'...",{'wikidata_id': 'Q11989'},9. Senfoni'de kaç bölüm var?,Senfonik No. 9'da kaç hareket var?,1.00,0.00,Symphony No. 9,9. Senfoni,9. Senfoni'de kaç hareket var?


In [23]:
# Save to TSV 
# trans_df.to_csv(TRANS_FILE, sep='\t', index=False)
df.to_csv(file,sep='\t',index=False)

In [50]:
# trans_sents = sentence_df["Cloud Translation"].tolist()

In [133]:
# Spanish BLEU 0.22304585226870685
# Italian BLEU 0.1595706884838815